In [ ]:
%run -i rename.py

In [ ]:
from models import Net2x, Net3x, Net4x, NetDN
models = [
    # model Class, kwargs, root, *paths
    (Net2x, {}, 0, '../model/a2/model_new.pth', '../model/p2/model_new.pth'),
    (Net3x, {}, 0, '../model/a3/model_new.pth', '../model/p3/model_new.pth'),
    (Net4x, {}, 0, '../model/a4/model_new.pth', '../model/p4/model_new.pth'),
    (NetDN, {}, 0, '../model/dn_lite5/model_new.pth', '../model/dn_lite10/model_new.pth', '../model/dn_lite15/model_new.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'convt_F.\.',), None, None, '', '0.')
]

In [ ]:
from models import RRDBNet
models = [
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'scale': 2}, 'params_ema', '../model/gan/RealESRGAN_x2plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3}, 'params_ema', '../model/gan/RealESRGAN_x4plus.pth'),
    (RRDBNet, {'num_in_ch': 3, 'num_out_ch': 3, 'num_block': 6}, 'params_ema', '../model/gan/RealESRGAN_x4plus_anime_6B.pth')
]
subs = [
    (re.compile(r'(body.[\d]+.rdb[\d]+.)conv([\d]+)(\..*)'), lambda m: ''.join((m[1], 'conv.', str(int(m[2]) - 1), m[3])))
]

In [ ]:
from videoSR import modules
rules = [o['weight'] for o in modules.values()]
models = [
  (None, {}, 'params', '../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth')
]
rsts = [
    # match, replace, with, suffix, insert
    ((r'[^\.]+_trunk\.',), 'main.', '', None, None),
]
subs = [
    # match, replace, with, suffix, insert
    (re.compile(r'(spynet\.basic_module\.[\d]+\.)basic_module\.(.*)'), r'\1\2'),
    (re.compile(r'upconv1\.(.*)'), r'upsample.0.\1'),
    (re.compile(r'upconv2\.(.*)'), r'upsample.3.\1'),
    (re.compile(r'conv_hr\.(.*)'), r'upsample.6.\1'),
    (re.compile(r'conv_last\.(.*)'), r'upsample.8.\1')
]

In [ ]:
m = renameByRules(models, rsts=rsts, subs=subs)

In [ ]:
modelPath = '../model/vsr/IconVSR_Vimeo90K_BDx4-cfcb7e00.pth'
# {'{name}.*' => '{name}': {'*'} for name in rules}
namespaced = lambda w, rules:\
  dict((name, dict((key.removeprefix('{}.'.format(name)), w[key]) for key in getRoot(w, name))) for name in rules)

w = namespaced(w, rules)
pp(w)

for key, o in modules.items():
  m1 = o['f']()
  print(key, m1.load_state_dict(w[key]))
torch.save(w, modelPath, pickle_protocol=4)

In [ ]:
modelPath = '../model/ESTRNN/ESTRNN_C80B15_BSD_1ms8ms.tar'
o = torch.load(modelPath)
pp(o['state_dict'])

In [ ]:
from MPRNet import MPRNet
models = [
    # model Class, kwargs, root, *paths
    (MPRNet, dict(n_feat=96, scale_unetfeats=48, scale_orsnetfeats=32), 'state_dict', '../model/MPRNet/model_deblurring.pth'),
    (MPRNet, dict(n_feat=80, scale_unetfeats=48, scale_orsnetfeats=32), 'state_dict', '../model/MPRNet/model_denoising.pth'),
    (MPRNet, dict(n_feat=40, scale_unetfeats=20, scale_orsnetfeats=16), 'state_dict', '../model/MPRNet/model_deraining.pth')
]
rsts = [
    ((r'(.+\.)body\.',), 'body.', '', None, None),
    ((r'(.+\.)CA\.',), 'CA.', '3.', None, None),
]
sm1 = lambda s: str(int(s) - 1)
subs = [
    (re.compile(r'shallow_feat([\d]+)(\..*)'), lambda m: ''.join(('shallow_feat.', sm1(m[1]), m[2]))),
    (re.compile(r'(sam|concat)([\d])[\d](\..*)'), lambda m: ''.join((m[1], '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage3_orsnet\.orb([\d]+)(\..*)'), lambda m: ''.join(('stage3_orsnet.orb.', sm1(m[1]), m[2]))),
    (re.compile(r'stage3_orsnet\.up_(enc|dec)1\.up(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.1.0', m[2]))),
    (re.compile(r'stage3_orsnet\.up_(enc|dec)([\d]+)\.([\d]+)\.up(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.', m[2], '.', m[3], m[4]))),
    (re.compile(r'stage3_orsnet\.conv_(enc|dec)([\d]+)(\..*)'), lambda m: ''.join(('stage3_orsnet.conv_', m[1], '.', sm1(m[2]), '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage3_orsnet\.(.*)'), r'encoder.2.\1'),
    (re.compile(r'(stage[\d]+_encoder\.encoder_level[\d]+\.)([\d]+)(\..*)'), lambda m: ''.join((m[1], str(int(m[2]) + 1), m[3]))),
    (re.compile(r'stage([\d]+)_(encoder|decoder)\.(encoder|decoder)_level([\d]+)(\..*)'), lambda m: ''.join((m[2], '.', sm1(m[1]), '.', m[3], '.', sm1(m[4]), m[5]))),
    (re.compile(r'stage([\d]+)_encoder\.down([\d])[\d]\.down(\..*)'), lambda m: ''.join(('encoder.', sm1(m[1]), '.encoder.', m[2], '.0', m[3]))),
    (re.compile(r'stage([\d]+)_encoder\.(csff_enc|csff_dec)([\d]+)(\..*)'), lambda m: ''.join(('encoder.', sm1(m[1]), '.', m[2], '.', sm1(m[3]), m[4]))),
    (re.compile(r'([^\.]+\.)skip_attn([\d]+)(\..*)'), lambda m: ''.join((m[1], 'skip_attn.', sm1(m[2]), m[3]))),
    (re.compile(r'(stage[\d]+_decoder\.up)[\d]([\d])(\..*)'), lambda m: ''.join((m[1], '.', sm1(m[2]), m[3]))),
    (re.compile(r'stage([\d])_(encoder|decoder)(\..*)'), lambda m: ''.join((m[2], '.', sm1(m[1]), m[3])))
]

In [ ]:
for param in m.parameters():
  param.requires_grad_(False)
m = m.to(dtype=config.dtype(), device=config.device()).eval()
opt = Option(models[-1][-1])
opt.scale = 1
opt.padding = 5
opt.align = 8
opt.ramCoef = 1 / 10000.
opt.squeeze = lambda x: x.squeeze(0)
opt.unsqueeze = lambda x: x.unsqueeze(0)
opt.modelCached = m

In [ ]:
original = readPic('../images/example1sn.png')
show(original)

In [ ]:
y = ensemble(opt)(original)
show(y)